In [1]:
# Load .env before anything else
from dotenv import load_dotenv
load_dotenv()

# LLM Setup
from langchain_groq import ChatGroq
import os

llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama-3.3-70b-versatile" # ✅ Correct Groq model name
)


In [2]:
# Let's build the research agent tools
from langchain_tavily import TavilySearch
from langchain.tools import tool
from exploit_info import get_cve_info
from web_scrape import scrape_web_pages

# Tool 1 — Tavily Search Tool
web_search = TavilySearch(max_results=3)

# Tool 2 — CVE info
@tool
def exploit_search(query: str) -> str:
    """Use this to search CVEs on NVD for a certain service."""
    return get_cve_info(query)

# Tool 3 — Web scraper
@tool
def web_scraper(query: list[str]) -> str:
    """Scrape and return page contents using LangChain's WebBaseLoader."""
    return str(scrape_web_pages(query))


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langgraph.prebuilt import create_react_agent

research_agent = create_react_agent(
    model=llm,
    tools=[web_search, web_scraper, exploit_search],
    prompt=(
        "You are a Penetration Testing research agent.\n\n"
        "INSTRUCTIONS:\n"
        "You have three tools for web search, CVE lookups, and scraping URLs.\n"
        "After you're done with your tasks, respond to the supervisor directly.\n"
        "Respond ONLY with the results of your work. Do NOT include ANY other text."
    ),
    name="research_agent",
)


In [4]:
from kali import connected_kali

@tool
def kali_linux(query: str) -> str:
    """Run Kali Linux terminal commands for pentesting or CTFs."""
    return connected_kali(query)

terminal_agent = create_react_agent(
    model=llm,
    tools=[kali_linux],
    prompt=(
        "You are a Terminal Agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Run Kali Linux CLI tools.\n"
        "- If tools are not installed, use: apt install <tool_name>\n"
        "- Respond ONLY with command results. Do NOT include any extra text."
    ),
    name="terminal_agent",
)


In [5]:
from decode_encode import convert_from_b64, convert_to_b64, detect_base, convert_number

@tool
def convert_from_base64(query: str) -> str:
    """Convert from base64."""
    return convert_from_b64(query)

@tool
def convert_to_base64(query: str) -> str:
    """Convert to base64."""
    return convert_to_b64(query)

@tool
def number_system_id(query: str) -> str:
    """Detect if the string is hex, binary, or decimal."""
    return detect_base(query)

@tool
def number_system_converter(value: str, source_format: str, target_format: str) -> str:
    """Convert between hex, binary, and decimal."""
    return convert_number(value, source_format, target_format)

decrypt_agent = create_react_agent(
    model=llm,
    tools=[convert_from_base64, convert_to_base64, number_system_id, number_system_converter],
    prompt=(
        "You are the Decryption Agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Decode base64, encode to base64, and convert number systems.\n"
        "- Respond ONLY with the results. Do NOT include extra text."
    ),
    name="decrypt_agent",
)


In [6]:
from langgraph_supervisor import create_supervisor

supervisor_graph = create_supervisor(
    model=llm,
    agents=[research_agent, terminal_agent, decrypt_agent],
    prompt=(
        "You are a powerful pentest supervisor managing 3 agents:\n"
        "- Research agent: Handles research and CVE lookups\n"
        "- Terminal agent: Executes Kali Linux CLI tools\n"
        "- Decrypt agent: Handles encoding/decoding & number conversions\n"
        "Assign tasks one at a time. Do not work yourself.\n"
    ),
    add_handoff_messages=True,
    add_handoff_back_messages=True,
    output_mode="full_history",
)

supervisor = supervisor_graph.compile()


In [7]:
result = supervisor.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Search for me about Ruto latest visit"
        }
    ]
})

# Print the full interaction history
for step in result:
    print(step)


Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel is_last_step, ignoring it.
Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel remaining_steps, ignoring it.


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jre525rgfbh968tqqq54ncwa` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 91578, Requested 11725. Please try again in 47m33.011s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}